In [19]:
from deepeval.metrics import AnswerRelevancyMetric,ContextualPrecisionMetric
from deepeval.evaluate import DisplayConfig
from deepeval.models import GeminiModel
from deepeval.test_case import LLMTestCase
from dotenv import load_dotenv
from deepeval import evaluate
import os

load_dotenv()

True

In [24]:
def execute():

    model = GeminiModel("gemini-2.5-flash",api_key=os.getenv('LLM_API_KEY'))

    answer_relevancy = AnswerRelevancyMetric(model=model,threshold=0.8)
    contextual_precision = ContextualPrecisionMetric(model=model,threshold=0.8)

    test_case1 = LLMTestCase(
        input="Explain the difference between supervised and unsupervised learning with examples.",
        actual_output="""Supervised learning uses labeled data to train models that can predict outcomes — for instance, predicting exam scores based on study hours.
    Unsupervised learning, on the other hand, works without labeled outputs, finding hidden patterns, like grouping news articles by topic using clustering.""",
        retrieval_context=["""Supervised learning uses labeled datasets to train models, meaning the input data is paired with the correct output. Common algorithms include linear regression and decision trees.  
    Unsupervised learning works with unlabeled data, aiming to find hidden patterns or structures. Examples include clustering (like K-means) and dimensionality reduction (like PCA).
    """],
        expected_output="""Supervised learning involves training a model on labeled data, where the correct outputs are known. For example, predicting house prices using historical data.
    Unsupervised learning deals with unlabeled data to find patterns or groupings, such as using K-means clustering to group customers by purchasing behavior."""
    )

    test_case2 = LLMTestCase(
        input="Explain the difference between supervised and unsupervised learning with examples.",
        actual_output="""Supervised learning is when humans supervise the AI, while unsupervised learning happens automatically.
    Supervised is better because it uses feedback, while unsupervised doesn’t.""",
        retrieval_context=["""Supervised learning uses labeled data with known outputs.  
    Unsupervised learning uses unlabeled data to discover hidden patterns or structures.
    """],
        expected_output="""Supervised learning uses labeled data to predict outcomes (e.g., predicting house prices).
    Unsupervised learning finds patterns in unlabeled data (e.g., customer segmentation using clustering)."""
    )

    results = evaluate([test_case1,test_case2], metrics=[answer_relevancy, contextual_precision], display_config=DisplayConfig(verbose_mode=False,print_results=False,show_indicator=False))

    return results

In [ ]:
results = execute()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001B1B6F269D0>


⚠ WARNING: No hyperparameters logged.
» ]8;id=952942;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 10.44s | token cost: 0.0 USD)
» Test Results (2 total tests):
   » Pass Rate: 100.0% | Passed: 2 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [57]:
import json
with open('./result.json','w') as f:
    json.dump(results.json(),fp=f)

C:\Users\D\AppData\Local\Temp\ipykernel_8440\4289594953.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(results.json(),fp=f)


In [26]:
def extract_test_results(data):

    extracted = []

    try:

        for test in data:
            test_info = {
                "test_name": test.name,
                "success": test.success,
                "input": test.input,
                "expected_output": test.expected_output,
                "actual_output": test.actual_output,
                "retrieval_context": test.retrieval_context,
                "metrics": []
            }

            for metric in test.metrics_data:
                metric_info = {
                    "metric_name": metric.name,
                    "score": metric.score,
                    "threshold": metric.threshold,
                    "success": metric.success,
                    "reason": metric.reason,
                    "evaluation_model": metric.evaluation_model
                }
                test_info["metrics"].append(metric_info)

            extracted.append(test_info)

    except Exception as e:

        print(f"Exception: {e}")

    return extracted

In [38]:
parsedResults = extract_test_results(data=results.test_results)

parsedResults

[{'test_name': 'test_case_0',
  'success': True,
  'input': 'Explain the difference between supervised and unsupervised learning with examples.',
  'expected_output': 'Supervised learning involves training a model on labeled data, where the correct outputs are known. For example, predicting house prices using historical data.\n    Unsupervised learning deals with unlabeled data to find patterns or groupings, such as using K-means clustering to group customers by purchasing behavior.',
  'actual_output': 'Supervised learning uses labeled data to train models that can predict outcomes — for instance, predicting exam scores based on study hours.\n    Unsupervised learning, on the other hand, works without labeled outputs, finding hidden patterns, like grouping news articles by topic using clustering.',
  'retrieval_context': ['Supervised learning uses labeled datasets to train models, meaning the input data is paired with the correct output. Common algorithms include linear regression and

In [30]:
from rich.console import Console
from rich.table import Table
from InquirerPy import inquirer

metrics = parsedResults[0]['metrics']

table = Table(title="Evaluation Metrics Summary")

table.add_column("Metric Name", justify="left", style="cyan", no_wrap=True)
table.add_column("Score", justify="center", style="green")
table.add_column("Threshold", justify="center", style="yellow")
table.add_column("Success", justify="center", style="bold")
table.add_column("Model", justify="left", style="magenta")
table.add_column("Reason", justify="left", style="white")

for metric in metrics:
    
    table.add_row(
        metric["metric_name"],
        str(metric["score"]),
        str(metric["threshold"]),
        "✅" if metric["success"] else "❌",
        metric["evaluation_model"],
        metric["reason"]
    )

Console().print(table)

                                            Evaluation Metrics Summary                                             
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metric Name          ┃ Score ┃ Threshold ┃ Success ┃ Model            ┃ Reason                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Answer Relevancy     │  1.0  │    0.8    │   ✅    │ gemini-2.5-flash │ The score is 1.00 because the output    │
│                      │       │           │         │                  │ perfectly addresses the input without   │
│                      │       │           │         │                  │ any irrelevant statements. Great job!   │
│ Contextual Precision │  1.0  │    0.8    │   ✅    │ gemini-2.5-flash │ The score is 1.00 because the first     │
│                      │       │           │         │                  │ node in the retrieval contexts is       │
│                      │       │           │         │                  │ perfectly relevant, directly explaining │
│                      │       │           │         │                  │ both supervised and unsupervised        │
│                      │       │           │         │                  │ learning, stating it "uses labeled      │
│                      │       │           │         │                  │ datasets to train models, meaning the   │
│                      │       │           │         │                  │ input data is paired with the correct   │
│                      │       │           │         │                  │ output," and unsupervised learning,     │
│                      │       │           │         │                  │ noting it "works with unlabeled data,   │
│                      │       │           │         │                  │ aiming to find hidden patterns or       │
│                      │       │           │         │                  │ structures." It also provides examples  │
│                      │       │           │         │                  │ like "clustering (like K-means)" which  │
│                      │       │           │         │                  │ is used in the expected output. This    │
│                      │       │           │         │                  │ demonstrates excellent retrieval        │
│                      │       │           │         │                  │ quality!                                │
└──────────────────────┴───────┴───────────┴─────────┴──────────────────┴─────────────────────────────────────────┘

In [44]:
from statistics import mean

def evaluationSummary(test_results):

    # Dictionary to store aggregated metric stats
    metric_summary = {}

    for test in test_results:
        for metric in test['metrics']:
            name = metric['metric_name']
            score = metric['score']
            success = metric['success']

            if name not in metric_summary:
                metric_summary[name] = {
                    'scores': [],
                    'success_count': 0,
                    'total_count': 0,
                    'threshold': metric.get('threshold', None)
                }

            metric_summary[name]['scores'].append(score)
            metric_summary[name]['total_count'] += 1
            if success:
                metric_summary[name]['success_count'] += 1

    # =============================
    # Aggregate Results
    # =============================

    overall_metrics = []
    for name, stats in metric_summary.items():
        avg_score = mean(stats['scores'])
        success_rate = stats['success_count'] / stats['total_count']
        overall_metrics.append({
            'metric_name': name,
            'average_score': round(avg_score, 3),
            'success_rate': f"{success_rate*100:.1f}%",
            'threshold': stats['threshold']
        })

    # =============================
    # Compute Overall Success Rate
    # =============================
    overall_success_rate = (
        sum(1 for t in test_results if t['success']) / len(test_results)
    )
    average_metric_score = mean(m['average_score'] for m in overall_metrics)

    return (overall_metrics,overall_success_rate,average_metric_score)

In [45]:
summary = evaluationSummary(parsedResults)

print(summary)

([{'metric_name': 'Answer Relevancy', 'average_score': 1.0, 'success_rate': '100.0%', 'threshold': 0.8}, {'metric_name': 'Contextual Precision', 'average_score': 1.0, 'success_rate': '100.0%', 'threshold': 0.8}], 1.0, 1.0)


In [60]:
import json

with open('result.json','r') as f:

    res = json.loads(json.load(f))

In [61]:
res

{'test_results': [{'name': 'test_case_0',
   'success': True,
   'metrics_data': [{'name': 'Answer Relevancy',
     'threshold': 0.8,
     'success': True,
     'score': 1.0,
     'reason': 'The score is 1.00 because the output perfectly addresses the input without any irrelevant statements. Great job!',
     'strict_mode': False,
     'evaluation_model': 'gemini-2.5-flash',
     'error': None,
     'evaluation_cost': 0.0,
     'verbose_logs': 'Statements:\n[\n    "Supervised learning uses labeled data to train models that can predict outcomes.",\n    "An example is predicting exam scores based on study hours.",\n    "Unsupervised learning works without labeled outputs.",\n    "Unsupervised learning finds hidden patterns.",\n    "An example is grouping news articles by topic using clustering."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "yes",\n       